### Notebook to calculate Pearson coefficient from real data using both time and temperature amplitude ratio, to see whether 1D hypothesis is actually relevant or not.

In [ ]:
#We get the data for point 46 (representing the highest k value, ie the most advective case)

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from pyheatmy import *
from pyheatmy.time_series_multiperiodic import *
import os as os
import csv as csv 

In [ ]:


#Création de l'objet time_series_multiperiodic

dt = 15*NSECINMIN
depth_sensor = DEFAULT_sensor_depth

ts = time_series_multiperiodic('ts')

df = ts.read_csv(52)
df.head()
dates = df['Date_heure']
data_temperature = array = df.drop(columns=['Date_heure']).to_numpy()

ts.values_time_series(dates, data_temperature, depth_sensors=depth_sensor)
ts.plot_temp()
ts.plot_pearson_coef()

# from 113
def plot_data(point_number, verbose = True):
    url = 'dataAnalysis\data_traite\point' + str(point_number) + '_temperature_traité.csv'
    df = pd.read_csv(url)
    df.columns = ['Date_heure', 'T_Sensor0', 'T_Sensor1', 'T_Sensor2', 'T_Sensor3']
    df['Date_heure'] = pd.to_datetime(df['Date_heure'], dayfirst = True)

    # Tracer toutes les colonnes sauf 'Date_heure' en fonction de 'Date_heure'
    df.set_index('Date_heure').plot(figsize=(10, 6), grid=True, title="Graphique des valeurs en fonction des dates")
    plt.xlabel("Date")
    plt.ylabel("Température (°C)")
    plt.title("signal de température pour le capteur " + str(point_number))
    plt.show()
